PROJECT SANTOSH KUMAR

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col,row_number, dense_rank ,rank
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
data = [("James","Sales",3000),
        ("Michael","Sales",4600),
      ("Robert","Sales",4100),
      ("Maria","Finance",3000),
      ("Raman","Finance",3000),
      ("Scott","Finance",3300),
      ("Jen","Finance",3900),
      ("Jeff","Marketing",3000),
      ("Kumar","Marketing",2000)]


schema =StructType([
    StructField("Name",StringType(),True),
    StructField("Department",StringType(),True),
    StructField("Salary",IntegerType(),True)
])

df = spark.createDataFrame(data,schema)
df.display()

Name,Department,Salary
James,Sales,3000
Michael,Sales,4600
Robert,Sales,4100
Maria,Finance,3000
Raman,Finance,3000
Scott,Finance,3300
Jen,Finance,3900
Jeff,Marketing,3000
Kumar,Marketing,2000


#2nd Highest salary By each department

## ROW NUMBER

In [0]:
Window_Dept = Window.partitionBy("Department").orderBy(col("Salary").desc())
df2=df.withColumn("row_number",row_number().over(Window_Dept))
df2 = df2.filter(col("row_number") == 2)\
         .filter(col("Department")=='Finance')

df3=df2.select("Name","Department","Salary")

df3.show()

+-----+----------+------+
| Name|Department|Salary|
+-----+----------+------+
|Scott|   Finance|  3300|
+-----+----------+------+



DROP THE COLUMN 
df2=df2.drop("row_number")

In [0]:
df2=df2.drop("row_number")
df2.show()

+-----+----------+------+
| Name|Department|Salary|
+-----+----------+------+
|Scott|   Finance|  3300|
+-----+----------+------+



##  DENSE_RANK


In [0]:
Window_Dense=Window.partitionBy("Department").orderBy(col("Salary").desc())

df_denserank=df.withColumn("Dense_Rank",dense_rank().over(Window_Dense))
# Add a new column with dense ranks within each department
df_denserank = df.withColumn("Dense_Rank", dense_rank().over(Window_Dense))


# Show the result
df_denserank.display()


Name,Department,Salary,Dense_Rank
Jen,Finance,3900,1
Scott,Finance,3300,2
Maria,Finance,3000,3
Raman,Finance,3000,3
Jeff,Marketing,3000,1
Kumar,Marketing,2000,2
Michael,Sales,4600,1
Robert,Sales,4100,2
James,Sales,3000,3


Databricks visualization. Run in Databricks to view.

Databricks data profile. Run in Databricks to view.

## Filter Highest SALARY Using DENSE_RANK 

In [0]:
df_filter= df_denserank.filter(col("Dense_Rank")==1)
df_filter.show()

+-------+----------+------+----------+
|   Name|Department|Salary|Dense_Rank|
+-------+----------+------+----------+
|    Jen|   Finance|  3900|         1|
|   Jeff| Marketing|  3000|         1|
|Michael|     Sales|  4600|         1|
+-------+----------+------+----------+



#HOW TO USE RANK FUNCTION IN DATABRICKS 

In [0]:
Window_Dense=Window.partitionBy("Department").orderBy(col("Salary").desc())

df_rank=df.withColumn("Rank",rank().over(Window_Dense))
# Add a new column with dense ranks within each department



# Show the result
df_rank.show()

+-------+----------+------+----+
|   Name|Department|Salary|Rank|
+-------+----------+------+----+
|    Jen|   Finance|  3900|   1|
|  Scott|   Finance|  3300|   2|
|  Maria|   Finance|  3000|   3|
|  Raman|   Finance|  3000|   3|
|   Jeff| Marketing|  3000|   1|
|  Kumar| Marketing|  2000|   2|
|Michael|     Sales|  4600|   1|
| Robert|     Sales|  4100|   2|
|  James|     Sales|  3000|   3|
+-------+----------+------+----+



## Filter 3rd Highest salary by each department by Rank function 

In [0]:
df_filter_rank= df_rank.filter(col("Rank")==3)
df_filter_rank.show()

+-----+----------+------+----+
| Name|Department|Salary|Rank|
+-----+----------+------+----+
|Maria|   Finance|  3000|   3|
|Raman|   Finance|  3000|   3|
|James|     Sales|  3000|   3|
+-----+----------+------+----+



## Group By sum of salary 

In [0]:
from pyspark.sql.functions import sum

df_sum = df.groupBy('Department') \
       .agg(sum('salary').alias('total_salary'))
df_sum.display()

Department,total_salary
Sales,11700
Finance,13200
Marketing,5000


In [0]:
         ,  eac.__analysis_date
                ,  cp.ssc
                ,  cp.profile_class
          from testing.ashpreet.elec_eac eac
          
          left 
          join  electricityconfigurationperiod as cp
          on    cp.meter_point_id = eac.meter_point_id
          and   eac.eac_effective_from_date between cp.effective_from_date and cp.effective_to_date
          
          where eac.__analysis_date = '{analysis_date_str}'

          """)
df_eac = df_eac.groupby("meter_point_id").apply(process_data)

# Grouped the ssc column, aggregate average of eac, count eac and filtered less than 10 and filtered ssc where not null.
result = df_eac.groupBy("ssc", "__analysis_date").agg(avg('eac').alias('Avg_eac'),count('eac').alias('eac_count')) \
           .filter("eac_count> 10") \
           .filter(df_eac.ssc.isNotNull())\
            .orderBy("Avg_eac")
        
# Displaying the result
result.display()
result.schema

# Save the DataFrame as a table
result.write \
    .mode("overwrite") \
    .partitionBy("__analysis_date") \
    .option("partitionOverwriteMode", "dynamic") \
    .saveAsTable(TARGET_TABLE)